In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
train_path = '../data/hy_round1_train_20200102'
test_path = '../data/hy_round1_testB_20200221'

In [3]:
train_df_list = []
for file_name in os.listdir(train_path):
    df = pd.read_csv(os.path.join(train_path, file_name))
    train_df_list.append(df)
    
test_df_list = []
for file_name in os.listdir(test_path):
    df = pd.read_csv(os.path.join(test_path, file_name))
    test_df_list.append(df)

In [4]:
train_df = pd.concat(train_df_list)
test_df = pd.concat(test_df_list)

In [5]:
train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

In [6]:
from tsfresh import extract_features

In [7]:
all_df = pd.concat([train_df, test_df], sort=False)

In [8]:
df = all_df.drop(columns=['type'])
y = all_df['type']

In [9]:
extracted_df = extract_features(df, column_id='渔船ID', column_sort='time')

Feature Extraction: 100%|██████████| 30/30 [57:02<00:00, 114.10s/it] 


In [10]:
train_df = extracted_df.iloc[:7000]
test_df = extracted_df.iloc[7000:]

In [11]:
y = []
for name, group in all_df.groupby('渔船ID'):
    y.append(group.iloc[0]['type'])

In [12]:
y_train = y[:7000]
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [13]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(train_df)
filtered_train_df = select_features(train_df, y_train)
filtered_test_df = test_df[filtered_train_df.columns]

/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/pandas/core/generic.py:9276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cond, other, inplace, axis, level, errors=errors, try_cast=try_cast


In [106]:
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score

def rmsle_cv(model=None, X_train=None, y_train=None):
    seed = 42
    n_folds = 5
    kf = KFold(n_folds, shuffle=True, random_state=seed).get_n_splits(X_train)
    rmse= cross_val_score(model, X_train, y_train,
                           scoring="f1_macro", cv = kf)
    return (rmse)

In [107]:
lc = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)

In [108]:
cv_result = rmsle_cv(lc, filtered_train_df.values, y_train)

In [109]:
np.mean(cv_result), np.std(cv_result)

(0.8543892133723275, 0.007716674544948446)

In [110]:
lc.fit(filtered_train_df.values, y_train)
result = lc.predict(filtered_test_df.values)

In [111]:
result = le.inverse_transform(result)
pd.DataFrame(result, index=filtered_test_df.index).to_csv('result.csv', header=None)

In [16]:
filtered_train_df['type'] = le.inverse_transform(y_train)

In [17]:
filtered_train_df.to_csv('train.csv')
filtered_test_df.to_csv('testB.csv')